In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/ara_proje_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/benign/aug_0_5669.png  
  inflating: train/benign/aug_0_6396.png  
  inflating: train/benign/aug_0_821.png  
  inflating: train/benign/aug_0_4940.png  
  inflating: train/benign/aug_0_10.png  
  inflating: train/benign/aug_0_9248.png  
  inflating: train/benign/aug_0_6968.png  
  inflating: train/benign/aug_0_6599.png  
  inflating: train/benign/aug_0_6012.png  
  inflating: train/benign/aug_0_2732.png  
  inflating: train/benign/aug_0_6190.png  
  inflating: train/benign/aug_0_824.png  
  inflating: train/benign/aug_0_9575.png  
  inflating: train/benign/aug_0_9135.png  
  inflating: train/benign/aug_0_7581.png  
  inflating: train/benign/aug_0_9464.png  
  inflating: train/benign/aug_0_3309.png  
  inflating: train/benign/aug_0_8782.png  
  inflating: train/benign/aug_0_6508.png  
  inflating: train/benign/aug_0_5852.png  
  inflating: train/benign/aug_0_783.png  
  inflating: train/benign/aug_0_3259.png  
  inflat

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import NasNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# --- 1. PARAMETRELER VE YOL TANIMLAMALARI ---
DATA_ROOT = "/content/" # Olası kök dizin
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = 3 # Örn: Benign, Sınıf A, Sınıf B (Sınıf sayınızı doğrulayın)
EPOCHS = 50
LEARNING_RATE = 0.00001


In [ ]:
# --- 2. VERİ YÜKLEYİCİLERİ VE ÖN İŞLEME ---
# ResNet-50 ImageNet ağırlıkları kullanılmayacağı için sadece 1/255 normalizasyonu yeterlidir.
datagen = ImageDataGenerator(
    # Bu, pikselleri sadece 0-255'ten 0-1'e çevirmek yerine,
    # ImageNet ortalama ve std'ye göre normalizasyonu uygular.
    preprocessing_function=preprocess_input
)

train_generator = datagen.flow_from_directory(
    DATA_ROOT + 'train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical' # Veya binary
)

valid_generator = datagen.flow_from_directory(
    DATA_ROOT + 'valid',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    DATA_ROOT + 'test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

Found 7031 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Found 115 images belonging to 3 classes.


In [ ]:
# --- 3. MODEL TANIMI (TL OLMADAN) ---

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout

# 1. CALLBACK'LERİN TANIMLANMASI
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,                 # 5 epoch boyunca ilerleme yoksa dur
    restore_best_weights=True,  # Durduktan sonra en iyi ağırlıkları geri yükle
    verbose=1
)

checkpoint_filepath = '/tmp/best_weights.weights.h5'
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

base_model = ResNet50(weights='imagenet',
                      include_top=False,
                      pooling='avg',
                      classifier_activation='softmax',
                      input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

base_model.trainable = False
x = base_model.output
x = Dropout(0.5)(x)

predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# --- 4. MODEL DERLEME VE EĞİTİM ---
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    # CALLBACK'ler buraya parametre olarak iletilir!
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.3110 - loss: 2.1720
Epoch 1: val_loss improved from inf to 1.18611, saving model to /tmp/best_weights.weights.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 50s 91ms/step - accuracy: 0.3110 - loss: 2.1716 - val_accuracy: 0.3917 - val_loss: 1.1861
Epoch 2/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3305 - loss: 1.7784
Epoch 2: val_loss improved from 1.18611 to 1.07585, saving model to /tmp/best_weights.weights.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 29s 66ms/step - accuracy: 0.3305 - loss: 1.7782 - val_accuracy: 0.4333 - val_loss: 1.0759
Epoch 3/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3807 - loss: 1.6005
Epoch 3: val_loss improved from 1.07585 to 0.97227, saving model to /tmp/best_weights.weights.h5


...

440/440 ━━━━━━━━━━━━━━━━━━━━ 30s 67ms/step - accuracy: 0.7470 - loss: 0.6000 - val_accuracy: 0.7583 - val_loss: 0.5340
Epoch 45/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7605 - loss: 0.5711
Epoch 45: val_loss improved from 0.53403 to 0.53325, saving model to /tmp/best_weights.weights.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 29s 66ms/step - accuracy: 0.7605 - loss: 0.5711 - val_accuracy: 0.7667 - val_loss: 0.5333
Epoch 46/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7714 - loss: 0.5563
Epoch 46: val_loss improved from 0.53325 to 0.53040, saving model to /tmp/best_weights.weights.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 29s 67ms/step - accuracy: 0.7713 - loss: 0.5563 - val_accuracy: 0.7667 - val_loss: 0.5304
Epoch 47/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7642 - loss: 0.5870
Epoch 47: val_loss improved from 0.53040 to 0.52809, saving model to /tmp/best_weights.weights.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 30s 69ms/step - accuracy: 0.7643 - loss: 0.5870 - val_accuracy: 0.7667 - val_loss: 0.5281
Epoch 48/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7797 - loss: 0.5469
Epoch 48: val_loss did not improve from 0.52809
440/440 ━━━━━━━━━━━━━━━━━━━━ 29s 65ms/step - accuracy: 0.7797 - loss: 0.5470 - val_accuracy: 0.7833 - val_loss: 0.5287
Epoch 49/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7747 - loss: 0.5531
Epoch 49: val_loss did not improve from 0.52809
440/440 ━━━━━━━━━━━━━━━━━━━━ 29s 66ms/step - accuracy: 0.7747 - loss: 0.5531 - val_accuracy: 0.7750 - val_loss: 0.5287
Epoch 50/50
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7710 - loss: 0.5494
Epoch 50: val_loss improved from 0.52809 to 0.52311, saving model to /tmp/best_weights.weights.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 30s 69ms/step - accuracy: 0.7710 - loss: 0.5494 - val_accuracy: 0.7750 - val_loss: 0.5231
Restoring model weights from the end of the best epoch: 50.


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [ ]:



valid_steps = int(np.ceil(valid_generator.samples / BATCH_SIZE))
y_pred_proba = model.predict(valid_generator, steps=valid_steps)
y_pred = y_pred_proba.argmax(axis=1)

y_true = valid_generator.classes

accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")

print("\n--- Tabloya Eklenecek Sonuçlar ---")
print(f"ResNet-50 TL Olmadan | Accuracy: {accuracy:.4f} | Recall: {recall:.4f} | Precision: {precision:.4f} | F1-Score: {f1:.4f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 595ms/step
Accuracy: 0.8750
Recall: 0.8600
Precision: 0.8693
F1-Score: 0.8643

--- Tabloya Eklenecek Sonuçlar ---
ResNet-50 TL Olmadan | Accuracy: 0.8750 | Recall: 0.8600 | Precision: 0.8693 | F1-Score: 0.8643


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# --- 1. TEST VERİ YÜKLEYİCİSİ ---
# data_root ve image_size'ın önceki kodlarda tanımlı olduğu varsayılır.
# DATA_ROOT = "/content/"
# IMAGE_SIZE = (224, 224)
# BATCH_SIZE = 16

datagen = ImageDataGenerator(rescale=1./255)

test_generator = datagen.flow_from_directory(
    DATA_ROOT + 'test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Test için karıştırma yapılmaz
)

# --- 2. TEST SETİ ÜZERİNDEN TAHMİN VE METRİK HESAPLAMA ---

print("\n--- Test Seti Üzerinden Metrikler Hesaplanıyor ---\n")

test_steps = int(np.ceil(test_generator.samples / BATCH_SIZE))
y_pred_proba_test = model.predict(test_generator, steps=test_steps)
y_pred_test = y_pred_proba_test.argmax(axis=1)
y_true_test = test_generator.classes

# Sınıf isimlerini al
class_labels = list(test_generator.class_indices.keys())

# Metrikler
accuracy_test = accuracy_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test, average='macro', zero_division=0)
precision_test = precision_score(y_true_test, y_pred_test, average='macro', zero_division=0)
f1_test = f1_score(y_true_test, y_pred_test, average='macro', zero_division=0)

print(f"Test Accuracy: {accuracy_test:.4f}")
print(f"Test Recall: {recall_test:.4f}")
print(f"Test Precision: {precision_test:.4f}")
print(f"Test F1-Score: {f1_test:.4f}")


# --- 3. KARMAŞIKLIK MATRİSİ (CONFUSION MATRIX) OLUŞTURMA ---

cm = confusion_matrix(y_true_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel('Tahmin Edilen Sınıf')
plt.ylabel('Gerçek Sınıf')
plt.title('ResNet-50  Test Karmaşıklık Matrisi')
plt.savefig('resnet_no_tl_confusion_matrix.png')
plt.close()

print("\nKarmaşıklık Matrisi (resnet_no_tl_confusion_matrix.png) oluşturuldu.")

Found 115 images belonging to 3 classes.

--- Test Seti Üzerinden Metrikler Hesaplanıyor ---



/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 481ms/step
Test Accuracy: 0.2783
Test Recall: 0.3385
Test Precision: 0.4240
Test F1-Score: 0.1526

Karmaşıklık Matrisi (resnet_no_tl_confusion_matrix.png) oluşturuldu.
